### Example of how to get rules dynamically for DLT (not used in the demo)

In [ ]:
%sql

CREATE OR REPLACE TABLE
  data_quality_dlt_dev.rules
AS SELECT
  col1 AS name,
  col2 AS constraint,
  col3 AS tag
FROM (
  VALUES
  ("client_id_not_null","client_id IS NOT NULL","clients"),
  ("address_valid_value","address in ('New York', 'California', 'Texas', 'Florida', 'Illinois')","clients"),
  ("product_id_not_null","product_id IS NOT NULL","products"),
  ("sales_id_not_null","sales_id IS NOT NULL","sales"),
  ("client_id_not_null","client_id IS NOT NULL","sales"),
  ("client_id_not_null","client_id IS NOT NULL","sales"),
  ("date_valid_range","date between '2020-01-01' and '2025-01-01'","sales"),
  ("quantity_valid_value","quantity > 0","sales")
)

In [ ]:
from pyspark.sql.functions import expr, col
def get_rules(tag):
  """
    loads data quality rules from a table
    :param tag: tag to match
    :return: dictionary of rules that matched the tag
  """
  rules = {}
  df = spark.read.table("data_quality_dlt_dev.rules")
  for row in df.filter(col("tag") == tag).collect():
    rules[row['name']] = row['constraint']
  return rules

In [ ]:
#@dlt.expect_all_or_drop(get_rules(table_name))
def temp_silver():
    return (
        dlt.readStream("bronze_data_quality")
        .withColumn("process_timestamp", F.current_timestamp())
)